# Conversational UI Chatbot App with ChatGPT, LangChain and Chainlit

Here we will build a advanced ChatGPT Conversational UI-based chatbot using LangChain and Chainlit with the following features:

- Custom Landing Page
- Conversational memory
- Result streaming capabilities (Real-time output)

## Install App and LLM dependencies

In [ ]:
!pip install langchain==0.3.11
!pip install langchain-openai==0.2.12
!pip install chainlit==1.3.2
!pip install pyngrok==7.2.2
pip install pydantic==2.10.1

## Load OpenAI API Credentials

Here we load it from a file so we don't explore the credentials on the internet by mistake

In [1]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
import yaml

with open('chatgpt_api_credentials.yml', 'r') as file:
    api_creds = yaml.safe_load(file)

In [ ]:
api_creds.keys()

dict_keys(['openai_key'])

In [ ]:
import os

os.environ['OPENAI_API_KEY'] = api_creds['openai_key']

## Write the app code here and store it in a py file

In [5]:
%%writefile app.py
# the following line is a magic command
# that will write all the code below it to the python file app.py
# we will then deploy this app.py file on the cloud server where colab is running
# if you have your own server you can just write the code in app.py and deploy it directly


from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferWindowMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain.schema.runnable.config import RunnableConfig
from langchain.schema import StrOutputParser
from operator import itemgetter
from dotenv import load_dotenv, find_dotenv
import chainlit as cl

@cl.on_chat_start
# this function is called when the app starts for the first time
async def when_chat_starts():

  # Load a connection to ChatGPT LLM
  load_dotenv('/home/santhosh/Projects/courses/Pinnacle/.env')
  chatgpt = ChatOpenAI(model_name='gpt-4o-mini', temperature=0.1,
                       streaming=True)

  # Add a basic system prompt for LLM behavior
  SYS_PROMPT = """
               Act as a helpful assistant and answer questions to the best of your ability.
               Do not make up answers.
               """

  # Create a prompt template for langchain to use history to answer user prompts
  prompt = ChatPromptTemplate.from_messages(
    [
      ("system", SYS_PROMPT),
      MessagesPlaceholder(variable_name="history"),
      ("human", "{input}"),
    ]
  )

  # Create a memory object to store conversation history window
  memory = ConversationBufferWindowMemory(k=20,
                                          return_messages=True)

  # Create a conversation chain
  conversation_chain = (
    RunnablePassthrough.assign(
      history=RunnableLambda(memory.load_memory_variables)
      |
      itemgetter("history")
    )
    |
    prompt
    |
    chatgpt
    |
    StrOutputParser() # to parse the output to show on UI
  )
  # Set session variables to be accessed when user enters prompts in the app
  cl.user_session.set("chain", conversation_chain)
  cl.user_session.set("memory", memory)


@cl.on_message
# this function is called whenever the user sends a prompt message in the app
async def on_user_message(message: cl.Message):

  # get the chain and memory objects from the session variables
  chain = cl.user_session.get("chain")
  memory = cl.user_session.get("memory")

  # this will store the response from ChatGPT LLM
  chatgpt_message = cl.Message(content="")

  # Stream the response from ChatGPT and show in real-time
  async for chunk in chain.astream(
    {"input": message.content},
    config=RunnableConfig(callbacks=[cl.LangchainCallbackHandler()]),
  ):
      await chatgpt_message.stream_token(chunk)
  # Finish displaying the full response from ChatGPT
  await chatgpt_message.send()
  # Store the current conversation in the memory object
  memory.save_context({"input": message.content},
                      {"output": chatgpt_message.content})


Overwriting app.py


## Start the app

In [5]:
!chainlit run app.py --port=8989 --watch &>./logs.txt

^C


If the above command doesn't work, use the below command.

In [7]:
!chainlit run app.py --port=8989

2024-12-14 18:15:46 - Your app is available at http://localhost:8989
2024-12-14 18:15:55 - Translated markdown file for en-US not found. Defaulting to chainlit.md.
/home/santhosh/Projects/courses/Pinnacle/Building End-to-End Generative AI Applications/Module 3 - Notebooks/app.py:42: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k=20,
2024-12-14 18:16:13 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
^C


In [26]:
from pyngrok import ngrok
import yaml

# Terminate open tunnels if exist
ngrok.kill()

# Setting the authtoken
# Get your authtoken from `ngrok_credentials.yml` file
with open('ngrok_credentials.yml', 'r') as file:
    NGROK_AUTH_TOKEN = yaml.safe_load(file)
ngrok.set_auth_token(NGROK_AUTH_TOKEN['ngrok_key'])

# Open an HTTPs tunnel on port XXXX which you get from your `logs.txt` file
ngrok_tunnel = ngrok.connect(8989)
print("Chainlit App:", ngrok_tunnel.public_url)

2024-07-24 15:06:27 - Killing ngrok process: 416532
2024-07-24 15:06:27 - Updating authtoken for default "config_path" of "ngrok_path": /home/santhosh/.config/ngrok/ngrok
2024-07-24 15:06:27 - Opening tunnel named: http-8989-efdb6608-bc91-4b94-96cb-70a937db1ef3
2024-07-24 15:06:28 - t=2024-07-24T15:06:28+0530 lvl=info msg="no configuration paths supplied"
2024-07-24 15:06:28 - t=2024-07-24T15:06:28+0530 lvl=info msg="using configuration at default config path" path=/home/santhosh/.config/ngrok/ngrok.yml
2024-07-24 15:06:28 - t=2024-07-24T15:06:28+0530 lvl=info msg="open config file" path=/home/santhosh/.config/ngrok/ngrok.yml err=nil
2024-07-24 15:06:28 - t=2024-07-24T15:06:28+0530 lvl=info msg="starting web service" obj=web addr=127.0.0.1:4040 allow_hosts=[]
2024-07-24 15:06:30 - t=2024-07-24T15:06:30+0530 lvl=info msg="client session established" obj=tunnels.session
2024-07-24 15:06:30 - t=2024-07-24T15:06:30+0530 lvl=info msg="tunnel session started" obj=tunnels.session
2024-07-24 1

## Change the Initial app screen

In [ ]:
%%writefile chainlit.md

# Welcome I am AI Assistant 🤖

How can I help you today?

Overwriting chainlit.md


## Remove running app processes

In [10]:
ngrok.kill()

In [ ]:
!ps -ef | grep app

In [12]:
!sudo kill -9 5617

[sudo] password for santhosh: 
